# IMPORTS

## Libraries

In [1]:
import math

import pandas as pd
import numpy  as np

from google.oauth2             import service_account
from googleapiclient.discovery import build



pd.set_option('display.max_columns', 200)

## Load Data

In [2]:
dfRaw1 = pd.read_feather('../Data/FeatherData/YouTubeStackExchangeQueryRaw.feather')

In [3]:
dfRaw1.sample(5)

,Id,Uploader,UploadDate,Title,Description,Categories,Tags,Duration,ViewCount,LikeCount,DislikeCount,AverageRating,Query,DaysSincePublication,ThumbnailUrl,ThumbnailWidth,ThumbnailHeight
1554,oShHO3XTubU,Learn Coding,2020-09-24,how to install anaconda(python) on windows 10 ...,"Hello guys, In this video i'm showing you How ...",People & Blogs,"anaconda navigator download and install, how t...",817.0,42,5.0,NaN,5.0,scipy,99.0,https://i.ytimg.com/vi/oShHO3XTubU/hqdefault.j...,168,94
2003,Bmya3yirFgs,Atom,2020-11-14,"MultiClass Logistic Regression, Python Machine...","In this #PythonMachineLearning series, #MultiC...",Education,"multiclass logistic regression, iris dataset, ...",1172.0,21,2.0,NaN,5.0,logistic,48.0,https://i.ytimg.com/vi/Bmya3yirFgs/hqdefault.j...,168,94
99,5KTdE-vQ1l8,Barki,2020-04-28,How to Install Scikit learn Library in Python|...,Hi Guys in this tutorial we learn how to install,Film & Animation,nan,157.0,216,10.0,NaN,5.0,scikit-learn,248.0,https://i.ytimg.com/vi/5KTdE-vQ1l8/hqdefault.j...,168,94
332,OPPS0fQdWF0,cctmexico,2018-04-15,Datos para entrenar | Training Data Set | Sci...,"Cómo utilizar SKlearn para separar tus datos, ...",Education,"Python, análisis de datos en Python, Python en...",1236.0,7,244.0,NaN,5.0,scikit-learn,992.0,https://i.ytimg.com/vi/OPPS0fQdWF0/hqdefault.j...,168,94
1346,mbSUVwXmHa0,Auribises,2020-03-25,Random Forest | Machine Learning | Python | Co...,Random Forest,Education,"machine learning, regression, random forest, c...",4014.0,152,4.0,NaN,5.0,random-forest,282.0,https://i.ytimg.com/vi/mbSUVwXmHa0/hqdefault.j...,168,94


# DESCRIPTION OF THE DATA

In [4]:
print(f'Number of Rows: {dfRaw1.shape[0]}')
print(f'Number of Columns: {dfRaw1.shape[1]}')

Number of Rows: 4382
Number of Columns: 17


## Fillout NA

In [5]:
dfRaw1.isnull().sum()

Id                         0
Uploader                   0
UploadDate                 0
Title                      0
Description              243
Categories                 0
Tags                       0
Duration                   0
ViewCount                  0
LikeCount                476
DislikeCount            2271
AverageRating            407
Query                      0
DaysSincePublication       0
ThumbnailUrl               0
ThumbnailWidth             0
ThumbnailHeight            0
dtype: int64

In [6]:
# LikeCount
dfRaw1['LikeCount'] = dfRaw1['LikeCount'].apply(lambda row: 0 if math.isnan(row) else row)

# DislikeCount
dfRaw1['DislikeCount'] = dfRaw1['DislikeCount'].apply(lambda row: 0 if math.isnan(row) else row)

# DislikeCount
dfRaw1['AverageRating'] = dfRaw1['AverageRating'].apply(lambda row: -1 if math.isnan(row) else row)

# Description
dfRaw1['Description'].fillna("no description", inplace=True)

In [7]:
dfRaw1.isnull().sum()

Id                      0
Uploader                0
UploadDate              0
Title                   0
Description             0
Categories              0
Tags                    0
Duration                0
ViewCount               0
LikeCount               0
DislikeCount            0
AverageRating           0
Query                   0
DaysSincePublication    0
ThumbnailUrl            0
ThumbnailWidth          0
ThumbnailHeight         0
dtype: int64

# Shuffle Data and Prepare to send to Google Sheets

In [8]:
dfShuffle = dfRaw1.sample(frac=1)
dfShuffle['UploadDate'] = dfShuffle['UploadDate'].astype(str)
dfShuffle = dfShuffle.values.tolist()

# Send to Google Sheets

In [10]:
# Documentation: https://developers.google.com/sheets/api/quickstart/python
SERVICE_ACCOUNT_FILE = 'D:/01-DataScience/04-Projetos/00-Git/Youtube-Video-Recommendations/Credentials/keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

credentials = None
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


# The ID of spreadsheet.
SAMPLE_SPREADSHEET_ID = '1uCur7jOXuLnwuwfWgoBL8mvDDvchuLf-o0X-AnOxS7s'

service = build('sheets', 'v4', credentials=credentials)


# Call the Sheets API
sheet = service.spreadsheets()

# Read Values
# result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
#                             range="YoutubeScrapeSample!A1:B11").execute()

# values = result.get('values', [])

# Write Values
request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                               range="YoutubeScrapeSample!A2", valueInputOption="USER_ENTERED", body={"values":dfShuffle}).execute()

In [12]:
dfRaw1.columns

Index(['Id', 'Uploader', 'UploadDate', 'Title', 'Description', 'Categories',
       'Tags', 'Duration', 'ViewCount', 'LikeCount', 'DislikeCount',
       'AverageRating', 'Query', 'DaysSincePublication', 'ThumbnailUrl',
       'ThumbnailWidth', 'ThumbnailHeight'],
      dtype='object')